In [34]:
import xarray as xr
import numpy as np
import sys
import os

import matplotlib.pyplot as plt

In [155]:
def interpolate_nan(ds, dims=('lat', 'lon'), use_landmask=True, **kwargs):
    if isinstance(dims, str):
        dims = [dims]
    for dim in dims:
        ds = ds.interpolate_na(dim, use_coordinate=False, **kwargs)
    if use_landmask:
        land_mask = (xr.open_zarr('/scratch/hydrodl/data/bucket.zarr/mask/').data>0)
        #ds = ds.where(land_mask)
        ds.values[~land_mask.values] = np.nan
    return ds

In [149]:
%%timeit
interpolate_nan(ds).compute()

3.19 s ± 28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
%%timeit
interpolate_nan(ds, use_landmask=False).compute()

34.3 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [157]:
%%timeit
interpolate_nan(ds).compute()

2.36 s ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
ds.interpolate_na()

In [8]:
import xarray as xr
import numpy as np
import sys
import os

In [9]:
sys.path.append('/workspace/hydrodl/src/')
#from utils.cdo_wrappers import cdo_gridbox
#from utils.plotting import plot_map, subplots_robinson, plot_hexbin, set_share_axes
#from utils.pyutils import ProgressBar
#from utils.metrics import get_metrics, weighted_avg_and_std, global_cell_size, xr_quantile
from utils.parallel import parcall

In [10]:
from utils.data_postprocessing import postprocess

In [11]:
postprocess()

2020-07-06 16:00:14,204	INFO resource_spec.py:212 -- Starting Ray with 319.43 GiB memory available for workers and up to 140.91 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


2020-07-06 16:00:14 - Parallel execution of function `coords2samp` using 6 CPUs with 15 total runs.


2020-07-06 16:00:14,548	WARNING services.py:939 -- Redis failed to start, retrying now.
2020-07-06 16:00:14,865	INFO services.py:1181 -- View the Ray dashboard at localhost:8265
2020-07-06 16:00:15,820	WARNING worker.py:1047 -- Failed to unpickle the remote function 'utils.parallel.remote_fun' with function ID 8ec3fb0b1422e237f93b667cfe92195fffc37fce. Traceback:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/ray/function_manager.py", line 177, in fetch_and_register_remote_function
    function = pickle.loads(serialized_function)
ModuleNotFoundError: No module named 'utils'

2020-07-06 16:00:15,832	WARNING worker.py:1047 -- Failed to unpickle the remote function 'utils.parallel.remote_fun' with function ID 8ec3fb0b1422e237f93b667cfe92195fffc37fce. Traceback:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/ray/function_manager.py", line 177, in fetch_and_register_remote_function
    function = pickle.loads(serialized_fu

(pid=raylet) I0706 16:00:14.973152 77695 77695 store_runner.cc:44] Allowing the Plasma store to use up to 151.296GB of memory.
(pid=raylet) I0706 16:00:14.973237 77695 77695 store_runner.cc:58] Starting object store with directory /dev/shm and huge page support dismelted
(pid=raylet) I0706 16:00:14.974669 77695 77695 grpc_server.cc:74] ObjectManager server started, listening on port 41693.
(pid=raylet) I0706 16:00:14.979892 77695 77695 node_manager.cc:166] Initializing NodeManager with ID 38adaed3147ca4ae5e13cbac06a5cb3d38bef00c
(pid=raylet) I0706 16:00:14.980429 77695 77695 grpc_server.cc:74] NodeManager server started, listening on port 55476.
(pid=raylet) I0706 16:00:14.985289 77695 77695 service_based_accessor.cc:739] Received notification for node id = 38adaed3147ca4ae5e13cbac06a5cb3d38bef00c, IsAlive = 1
(pid=raylet) I0706 16:00:14.986171 77695 77695 service_based_accessor.cc:739] Received notification for node id = 38adaed3147ca4ae5e13cbac06a5cb3d38bef00c, IsAlive = 1


2020-07-06 16:00:15,980	WARNING worker.py:1047 -- Failed to unpickle the remote function 'utils.parallel.remote_fun' with function ID 8ec3fb0b1422e237f93b667cfe92195fffc37fce. Traceback:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/ray/function_manager.py", line 177, in fetch_and_register_remote_function
    function = pickle.loads(serialized_function)
ModuleNotFoundError: No module named 'utils'

2020-07-06 16:00:16,517	WARNING worker.py:1047 -- Failed to unpickle the remote function 'utils.parallel.remote_fun' with function ID 8ec3fb0b1422e237f93b667cfe92195fffc37fce. Traceback:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/ray/function_manager.py", line 177, in fetch_and_register_remote_function
    function = pickle.loads(serialized_function)
ModuleNotFoundError: No module named 'utils'



2020-07-06 16:00:18 - Done, elapsed time: 0 m 4 s (0 m 0 s per call).


RayTaskError(RuntimeError): [36mray::utils.parallel.remote_fun()[39m (pid=77708, ip=172.17.0.2)
  File "python/ray/_raylet.pyx", line 446, in ray._raylet.execute_task
RuntimeError: This function was not imported properly.

In [3]:
def add_msc_and_ano(ds, timescale='month', variables=['mod', 'obs'], label_months=True, add_yano=True):
    if variables == 'all':
        variables = list(ds.data_vars)
    for v in variables:
        msc = ds[v].groupby('time.' + timescale).mean('time', keep_attrs=True).compute()
        ano = (ds[v].groupby('time.' + timescale) - msc).compute()
        if timescale == 'month':
            if label_months:
                msc['month'] = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
            else:
                msc['month'] = np.arange(1, 13)
        ds[v + '_msc'] = msc
        ds[v + '_ano'] = ano
        if add_yano:
            yearly = ds[v].groupby('time.year').mean('time', keep_attrs=True).compute()
            yearly -= yearly.mean('year', keep_attrs=True).compute()
            ds[v + '_yano'] = yearly
        ds[v + '_ano'].attrs = ds[v].attrs
        ds[v + '_msc'].attrs = ds[v].attrs
        ds[v + '_yano'].attrs = ds[v].attrs
    return ds

def subset_region(ds, r, region=['Subtropic', 'TemperateNH', 'Tropic', 'ColdNH', 'SH']):
    """Subset dataset bt a regional mask.

    Sets all values outside regions to np.nan.

    Parameters
    ----------
    ds: xarray.DataArray or xarray.Dataset
        The dataset to subset.
    regions : str or list of str
        The region name to select. If `region` is a list, an xarray.Dataset with
        a new dimension corresponding to the region names will be returned. If
        region is a string, an xarray.Dataset is returned.
    r : xr.Dataset
        The regions.

    Returns
    -------
    xarray.Dataset or xarray.DataArray

    """
    return ds.where(r[region], np.nan)

text_box = dict(facecolor='white', edgecolor='none', pad=0, alpha=.8)

def plot_ts(ts, cr, path=None):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        if ts == 'raw':
            ts = ''
            label = 'monthly time-series'
        elif ts == 'ano':
            ts = '_ano'
            label = 'monthly anomalies'
        elif ts == 'msc':
            ts = '_msc'
            label = 'monthly seasonality'
        else:
            raise ValueError(f'ts `{ts}` not supported, use one of (`raw`, `msc`, `ano`)')

        fig, ax = plt.subplots(5, 5, figsize=(17, 8), sharey='row', sharex=True, gridspec_kw={'wspace': 0.04, 'hspace': 0})

        for i, (ds, var) in enumerate(zip([swe, et, q, tws], ['swe', 'et', 'q', 'tws'])):

            ax[i, 0].xaxis.set_tick_params(which='both', labelleft=True)

            units = f" [{ds.obs.attrs.get('units', '-')}]"

            time = ds.month.values if ts == '_msc' else ds.time.values

            ds = add_msc_and_ano(ds[['obs', 'mod']])

            obs_var = 'obs' + ts
            mod_var = 'mod' + ts

            for k, region in enumerate(regions):
                dsh = subset_region(ds, cr, region)
                dsh_m = dsh.mean(dim=('lat', 'lon'))
                dsh_q = dsh.quantile([0.2, 0.8], dim=('lat', 'lon'))

                dsh_m_obs = dsh_m[obs_var]
                dsh_m_mod = dsh_m[mod_var]

                cor = np.corrcoef(dsh_m_obs, dsh_m_mod)[0, 1]
                mef = (1 - np.sum(np.power(dsh_m_mod - dsh_m_obs, 2)) / np.sum(np.power(dsh_m_obs - np.mean(dsh_m_obs), 2))).values

                ax[i, k].fill_between(time, dsh_q[obs_var].isel(quantile=0), dsh_q[obs_var].isel(quantile=1), alpha=0.2, facecolor='k', label=r'$observed_{q0.2-0.8}$')
                ax[i, k].fill_between(time, dsh_q[mod_var].isel(quantile=0), dsh_q[mod_var].isel(quantile=1), alpha=0.4, facecolor='tab:blue', label=r'$modeled_{q0.2-0.8}$')

                ax[i, k].plot(time, dsh_m[mod_var], alpha=0.85, color='tab:blue', linewidth=2., label=r'$modeled_{mean}$')
                ax[i, k].plot(time, dsh_m[obs_var], alpha=1.0, color='k', linewidth=1.3, label=r'$observed_{mean}$', linestyle='dotted')

                ax[i, k].text(0.05, 0.95, f'r={cor:0.3f}, mef={mef:0.2f}', horizontalalignment='left', verticalalignment='top', transform=ax[i, k].transAxes, bbox=text_box)

                if i == 0:
                    ax[0, k].set_title(region, size=16)

                ax[i, k].tick_params(axis='x', rotation=45)

            ax[i, 0].set_ylabel(var.lower() + units, size=12)
                

        leg = ax[-1, 2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=4, fontsize=14)
        leg.set_title(label, prop={'size': 14, 'weight': 'heavy'})

    for ax_ in ax.flat:
        ax_.label_outer()
    for ax_ in ax[:-1, :].flat:
        ax_.xaxis.set_visible(False)
    for ax_ in ax[:, 1:].flat:
        ax_.yaxis.set_visible(False)
    if ts == '_msc':
        for ax_ in ax[-1, :].flat:
            for label in ax_.xaxis.get_ticklabels()[1::2]:
                label.set_visible(False)

    fig.patch.set_alpha(0)
    fig.align_ylabels(ax[:, 0])
    if path is not None:
        fig.savefig(path, dpi=300, bbox_inches='tight')

data_units= {
    'swe': 'mm',
    'et': 'mm d-1',
    'q': 'mm d-1',
    'tws': 'mm',
    'winp': 'mm d-1',
    'qf': 'mm d-1',
    'qb': 'mm d-1',
    'smrec': 'mm d-1',
    'gwrec': 'mm d-1',
    'gw': 'mm',
    'cwd': 'mm',
    'cwd_overflow': 'mm d-1',
    'qf_frac': '-',
    'smrec_frac': '-',
    'gwrec_frac': '-',
    'cwd_overflow_qf_frac': '-',
    'smelt': 'mm d-1',
    'sacc': 'mm d-1',
    'sfrac': '-',
    'ef': '-',
    'prec': 'mm d-1',
    'rn': 'w m-2 d-1',
    'tair': 'K'
}

cr = xr.open_dataset('/workspace/hydrodl/data/processed/1d/static/clusterRegions.nc')
regions = ['Subtropic', 'TemperateNH', 'Tropic', 'ColdNH', 'SH']
regions_lookup = {
    'SH': 5,
    'Tropic': 3,
    'Subtropic': 1,
    'TemperateNH': 2,
    'ColdNH': 4
}

def coords2samp(path, cr, write_file=True, data_vars=None, **kwargs):
    ds = xr.open_dataset(path, **kwargs)
    if data_vars:
        ds = ds[data_vars]
    mask_path = os.path.join(os.path.dirname(path), 'cv_masks.nc').replace('predictions_spinup', 'predictions')
    m = xr.open_dataset(mask_path).test
    ds['regions'] = cr.regions * m
    ds = ds.stack(sample=('lat', 'lon')).reset_index('sample')
    ds = ds.where(ds.regions > 0, drop=True)
    ds['sample'] = np.arange(len(ds.sample))
    #for k, v in data_units.items():
    #    ds[k].attrs['units']: v
    if write_file:
        ds.to_netcdf(path.replace('.nc', 'regions.nc'))
    else:
        return ds

class PathIter(object):
    def __init__(self, pred_path):
        self.pred_path = pred_path
        pass
    def iter_offsets_star(self):
        for offset in ['10', '01', '11']:
            path = f'{self.pred_path}pred_{offset}_*/{filename}'
            yield path
    def iter_offsets(self, filename):
        for offset in ['10', '01', '11']:
        #for offset in ['10', '01']:
            folds = []
            for fold in range(5):
                path = f'{self.pred_path}pred_{offset}_{fold}/{filename}'
                folds.append(path)
            yield folds
    def iter_paths(self):
        for offset in ['10', '01', '11']:
            for fold in range(5):
                path = f'{self.pred_path}pred_{offset}_{fold}/{filename}'
                yield path

def preproc(ds):

    offset_lookup = {
        '01': 0,
        '10': 1,
        '11': 2
    }

    filepath = ds.encoding['souegionsce']

    offset_fold = filepath.split('pred_')[1]
    offset = offset_lookup[offset_fold[:2]]
    fold = int(offset_fold[3])

    ds = ds.expand_dims(dim={'fold': [fold]})
    ds = ds.expand_dims(dim={'offset': [offset]})

    return ds

def preproc_aggr(ds):
    ds = preproc(ds)

    ds_new_list = []
    regions_list = []
    for r, r_i in regions_lookup.items():
        regions_list.append(r)
        ds_r = ds.where(ds.regions==r_i)
        ds_new = ds_r.mean('sample', keep_attrs=True)
        for var in data_units.keys():
            ds_new[var + '_q'] = xr_quantile(ds_r[var], [0.2, 0.5, 0.8], dim='sample')
        for k, v in data_units.items():
            ds_new[k].attrs['units']: v
            ds_new[k + '_q'].attrs['units']: v
        ds_new_list.append(ds_new)
    ds_combined = xr.combine_nested(ds_new_list, concat_dim='region')
    ds_combined['region'] = regions_list
    return ds_combined

In [ ]:
pred_path = '/scratch/hydrodl/experiments/hybrid/all_vars_task_weighting/cv/predictions/'

paths = []
i = 0
for offset_i, offset in enumerate(['10', '01', '11']):
    for fold in range(5):
        path = f'{pred_path}pred_{offset}_{fold}/daily_pred.nc'
        paths.append({'path': path})
        i += 1

pathiter = PathIter(pred_path)
parcall(coords2samp, paths, cr=cr, num_cpus=6, write_file=True, chunks={'time': 400});
ds = [xr.open_mfdataset(p, combine='by_coords', preprocess=preproc_aggr, chunks={'time': 400}) for p in pathiter.iter_offsets('daily_predregions.nc')]
xr.combine_by_coords(ds).to_netcdf(f'{pred_path}daily_metrics_combined.nc')

2020-07-05 14:26:41,350	INFO resource_spec.py:212 -- Starting Ray with 325.1 GiB memory available for workers and up to 143.34 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).


2020-07-05 14:26:41 - Parallel execution of function `coords2samp` using 6 CPUs with 15 total runs.


2020-07-05 14:26:41,599	WARNING services.py:939 -- Redis failed to start, retrying now.
2020-07-05 14:26:41,923	INFO services.py:1181 -- View the Ray dashboard at localhost:8265


(pid=raylet) I0705 14:26:41.977524 81106 81106 store_runner.cc:44] Allowing the Plasma store to use up to 153.91GB of memory.
(pid=raylet) I0705 14:26:41.977593 81106 81106 store_runner.cc:58] Starting object store with directory /dev/shm and huge page support dismelted
(pid=raylet) I0705 14:26:41.978834 81106 81106 grpc_server.cc:74] ObjectManager server started, listening on port 42833.
(pid=raylet) I0705 14:26:41.994899 81106 81106 node_manager.cc:166] Initializing NodeManager with ID da1cbc91fac8d32b52b388f82ceb368690e2c53a
(pid=raylet) I0705 14:26:41.995299 81106 81106 grpc_server.cc:74] NodeManager server started, listening on port 45847.
(pid=raylet) I0705 14:26:41.999208 81106 81106 service_based_accessor.cc:739] Received notification for node id = da1cbc91fac8d32b52b388f82ceb368690e2c53a, IsAlive = 1
2020-07-05 14:27:05 - Done, elapsed time: 0 m 24 s (0 m 1 s per call).


In [ ]:
pred_path = '/scratch/hydrodl/experiments/hybrid/all_vars_task_weighting/cv/predictions_spinup/'

paths = []
i = 0
for offset_i, offset in enumerate(['10', '01', '11']):
    for fold in range(5):
        path = f'{pred_path}pred_{offset}_{fold}/daily_pred.nc'
        paths.append({'path': path})
        i += 1

In [ ]:
pathiter = PathIter(pred_path)
parcall(coords2samp, paths, cr=cr, num_cpus=6, write_file=True, chunks={'time': 400});
ds = [xr.open_mfdataset(p, combine='by_coords', preprocess=preproc_aggr, chunks={'time': 400}) for p in pathiter.iter_offsets('daily_predregions.nc')]
# xr.combine_by_coords(ds).to_netcdf(f'{pred_path}daily_metrics_combined.nc')